In [13]:
import pandas as pd
import numpy as np
from pathlib import Path
import subprocess

To generate the anat_mri.txt file (necessary), run the following commands in a terminal:
```
cd ~/neurospin/acquisition/database/Prisma_fit
find -maxdepth 3 -type d -iname "mprage" > /home/is153802/code/formatting/anat_MRI/anat_mri.txt
chmod 777 /home/is153802/code/formatting/anat_MRI/anat_mri.txt
```

In [14]:
# Make your dictionary nip to sn 

modality = 'distraction'

if modality == 'distraction':
    dict_nip_to_sn = {'mn_240236': '2',
                    'to_220041': '4'}

In [15]:
# Where the anat will be stored before merging with the BIDS dataset
anat_base_path = '/home_local/Bonnaire/data'

# Where we put the temporary scans folders
output_folder = Path(f"{anat_base_path}/{modality}")

# Where the code is 
anat_code_path = '/home_local/Bonnaire/scripts'

# Where the scans folders are located
scan_path = Path('/neurospin/acquisition/database/Prisma_fit')

# Where we want to merge the two BIDS datasets
bids_folder = Path(f"/home_local/Bonnaire/data/LPP_MEG_{modality}")

subj = [key for key in dict_nip_to_sn]

subjects = [sub.replace('_','') for sub in subj]

list_scans = []
with open(f'{anat_code_path}/anat_mri.txt', 'r') as fh:
    for line in fh:
        list_scans.append((line))

list_scans = [scan.replace('\n','') for scan in list_scans]

list_scans_subjects = [scan[11:19] for scan in list_scans]

list_scans_subjects

['mn240236', 'to220041']

In [16]:
# Intersecting the subjects for LPP and the scans
list_scans_final = [scan for i, scan in enumerate(list_scans) if list_scans_subjects[i] in (subjects)]
list_scans_final

['./20240606/mn240236-8479_001/000002_mprage-sag-T1-160sl',
 './20240620/to220041-8503_001/000002_mprage-sag-T1-160sl']

In [17]:
# Get the number of unique patients

dict_pat_dat = {}

dict_pat_scan = {}

# Sort the list of scans in order to only get the latest MRI scans
list_scans_final.sort()

pat = np.array([pati[11:19] for pati in list_scans_final]) # list of participants
dates = np.array([pati[2:10] for pati in list_scans_final]) # list of dates of anat
scan_nb = np.array([pati[32:35] for pati in list_scans_final]) # list of dates of scan_nb
scan_nb_final = [(f"{int(scan_nb[i]):01}") for i in range(scan_nb.shape[0])]

for pa, da in zip(pat,dates):
    dict_pat_dat[pa] = da
    
for pa, scan in zip(pat,scan_nb_final):
    dict_pat_scan[pa] = scan

In [18]:
# For each subject:
# Get their subset of scans
# Sort it by date, and take the latest
scan_to_move = []

for sub in subjects:
    list_scan_specific = [scan for i, scan in enumerate(list_scans_final) if scan[11:19] == sub]
    # If no scan for the subject, skip it
    if list_scan_specific == []:
        continue
    # print(f'Sub {sub} ! list {list_scan_specific}')
    list_scan_specific.sort(reverse = True)


    scan_to_move.append((list_scan_specific[0]))
scan_to_move = [scan.replace('./','') for scan in scan_to_move]
print(scan_to_move)

['20240606/mn240236-8479_001/000002_mprage-sag-T1-160sl', '20240620/to220041-8503_001/000002_mprage-sag-T1-160sl']


In [19]:
for sub in output_folder.iterdir():
    print(str(sub)[-8:])

port.tsv


In [20]:
# Get the format of the previous participants_to_import.tsv and rebuild a new one updated
df__ = pd.read_csv(f'{anat_code_path}/{modality}/participants_to_import.tsv',delimiter='\t')
df = pd.DataFrame(index=[0],columns=range(8))
df.columns = df__.columns

dff = pd.DataFrame()
for sub in subjects:
    if any(sub in key for key in dict_pat_dat) and (sub in dict_pat_scan.keys()):
        sub_ = sub[:2] + '_' + sub[2:]
        date = dict_pat_dat[sub][:4] + '-' + dict_pat_dat[sub][4:6] + '-' + dict_pat_dat[sub][6:]
        df_= pd.DataFrame([f'sub-{dict_nip_to_sn[sub_]}', sub, {}, '01', date, None, 'prisma', f"(('{dict_pat_scan[sub]}','anat','T1w'))"])
        dff = pd.concat([dff,df_],axis = 1)
    else: 
        print(f'No anat for {sub} yet')
        
dff_t = dff.transpose()
dff_t.columns = df.columns
final = pd.concat([df,dff_t])
final = final.iloc[1:,:]
final

final.to_csv(f'./{modality}/participants_to_import.tsv',sep='\t',index= False)

In [21]:
final

,participant_id,NIP,infos_participant,session_label,acq_date,acq_label,location,to_import
0,sub-2,mn240236,{},01,2024-06-06,None,prisma,"(('2','anat','T1w'))"
0,sub-4,to220041,{},01,2024-06-20,None,prisma,"(('2','anat','T1w'))"
